In [1]:



import torch

# Check if CUDA is available
print("🚀 CUDA Available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("🔢 Number of GPUs:", torch.cuda.device_count())
    print("🎯 Current GPU Device:", torch.cuda.current_device())
    print("💻 GPU Name:", torch.cuda.get_device_name(torch.cuda.current_device()))
    print("⚙️ CUDA Version:", torch.version.cuda)
    print("🧩 cuDNN Version:", torch.backends.cudnn.version())
else:
    print("❌ CUDA not available. Running on CPU.")


🚀 CUDA Available: True
🔢 Number of GPUs: 1
🎯 Current GPU Device: 0
💻 GPU Name: NVIDIA GeForce RTX 3060
⚙️ CUDA Version: 12.4
🧩 cuDNN Version: 90100


In [2]:
import sys
sys.path.insert(0, r"F:\AudioSep\bandit")

from core.data.dnr.dataset import DivideAndRemasterRandomChunkDataset

ds = DivideAndRemasterRandomChunkDataset(
    data_root=r"F:\AudioSep\DnR\v2np",
    split="train",
    target_length=10,
    chunk_size_second=6.0,
    fs=44100,
    npy_memmap=True,
)

print(f"Dataset length: {len(ds)}")
sample = ds[0]
print("Track:", sample["track"])
print("Shapes:", {k: v.shape for k, v in sample["audio"].items()})


D:\anaconda\envs\ambi\lib\site-packages\lightning_fabric\__init__.py:41: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.


Dataset length: 10
Track: train/10408
Shapes: {'mixture': (1, 264600), 'speech': (1, 264600), 'music': (1, 264600), 'effects': (1, 264600), 'mne': (1, 264600)}


In [3]:
import sys
sys.path.insert(0, r"F:\AudioSep\bandit")

from core.data.dnr.datamodule import DivideAndRemasterDataModule

dm = DivideAndRemasterDataModule(
    data_root=r"F:\AudioSep\DnR\v2np",
    batch_size=2,
    num_workers=8,
    use_speech_reverb=False,
    train_kwargs=dict(target_length=20, chunk_size_second=6.0, fs=44100, npy_memmap=True),
    val_kwargs=dict(chunk_size_second=6.0, hop_size_second=3.0, fs=44100, npy_memmap=True),
    test_kwargs=dict(fs=44100, npy_memmap=True),
)

dm.setup("fit")
batch = next(iter(dm.train_dataloader()))
print(batch["track"])
print({k: v.shape for k, v in batch["audio"].items()})


['train/13937', 'train/13291']
{'mixture': torch.Size([2, 1, 264600]), 'speech': torch.Size([2, 1, 264600]), 'music': torch.Size([2, 1, 264600]), 'effects': torch.Size([2, 1, 264600]), 'mne': torch.Size([2, 1, 264600])}


In [5]:
import sys, inspect, importlib, io, os
sys.path.insert(0, r"F:\AudioSep\bandit")

import core
print("core loaded from:", core.__file__)

# Try to access the function directly
try:
    from core import parse_model_config
    print("--- parse_model_config source ---")
    print(inspect.getsource(parse_model_config))
except Exception as e:
    print("Direct import failed:", repr(e))
    # Fallback: read the file contents and print around the function
    p = core.__file__
    if p.endswith("__init__.pyc"):
        p = p[:-1]  # -> __init__.py
    print("\n--- core/__init__.py contents (first 400 lines) ---")
    with open(p, "r", encoding="utf-8") as f:
        text = f.read()
    print(text[:4000])  # show first chunk; if needed, search:
    print("\n--- Names I see that look like valid model keys ---")
    import re
    keys = set(re.findall(r'["\']([A-Za-z0-9_\-]+)["\']', text))
    # Heuristics for likely model keys
    for k in sorted([k for k in keys if k.lower().startswith("bandit") or k.lower() in {"bandit","demucs","htdemucs","unet","xumx"}]):
        print(" -", k)


core loaded from: F:\AudioSep\bandit\core\__init__.py
--- parse_model_config source ---
def parse_model_config(config: ConfigDict) -> Any:
    name = config["name"]

    for module in [model]:
        if name in module.__dict__:
            return module.__dict__[name](**config["kwargs"])

    raise NameError



In [6]:
import sys
sys.path.insert(0, r"F:\AudioSep\bandit")

from core import model
print("Available model classes in core.model:")
print([k for k,v in model.__dict__.items() if isinstance(v, type)])


Available model classes in core.model:
['MultiMaskMultiSourceBandSplitRNN', 'MultiMaskMultiSourceBandSplitTransformer', 'MultiMaskMultiSourceBandSplitConv', 'PatchingMaskMultiSourceBandSplitRNN', 'SingleMaskMultiSourceBandSplitRNN', 'SingleMaskMultiSourceBandSplitTransformer', 'IdealRatioMask', 'PhaseSensitiveFilter', 'IdealAmplitudeMask', 'IdealWienerMask', 'IdealBinaryMask', 'IdentityMask', 'OpenUnmixWrapper', 'HDemucsWrapper']


In [7]:
import sys, inspect
sys.path.insert(0, r"F:\AudioSep\bandit")
from core import model

print(inspect.getsource(model.SingleMaskMultiSourceBandSplitRNN))
# or:
print(inspect.signature(model.SingleMaskMultiSourceBandSplitRNN))


class SingleMaskMultiSourceBandSplitRNN(SingleMaskMultiSourceBandSplitBase):
    def __init__(
            self,
            in_channel: int,
            band_specs_map: Union[str, Dict[str, List[Tuple[float, float]]]],
            fs: int = 44100,
            require_no_overlap: bool = False,
            require_no_gap: bool = True,
            normalize_channel_independently: bool = False,
            treat_channel_as_feature: bool = True,
            n_sqm_modules: int = 12,
            emb_dim: int = 128,
            rnn_dim: int = 256,
            bidirectional: bool = True,
            rnn_type: str = "LSTM",
            mlp_dim: int = 512,
            hidden_activation: str = "Tanh",
            hidden_activation_kwargs: Optional[Dict] = None,
            complex_mask: bool = True,
            n_fft: int = 2048,
            win_length: Optional[int] = 2048,
            hop_length: int = 512,
            window_fn: str = "hann_window",
            wkwargs: Optional[Dict] = None,


In [2]:
import inspect
from core.model.bsrnn.wrapper import SingleMaskMultiSourceBandSplitRNN
print("INIT SIGNATURE:", inspect.signature(SingleMaskMultiSourceBandSplitRNN.__init__))

D:\anaconda\envs\ambi\lib\site-packages\lightning_fabric\__init__.py:41: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.


INIT SIGNATURE: (self, in_channel: int, band_specs_map: Union[str, Dict[str, List[Tuple[float, float]]]], fs: int = 44100, require_no_overlap: bool = False, require_no_gap: bool = True, normalize_channel_independently: bool = False, treat_channel_as_feature: bool = True, n_sqm_modules: int = 12, emb_dim: int = 128, rnn_dim: int = 256, bidirectional: bool = True, rnn_type: str = 'LSTM', mlp_dim: int = 512, hidden_activation: str = 'Tanh', hidden_activation_kwargs: Optional[Dict] = None, complex_mask: bool = True, n_fft: int = 2048, win_length: Optional[int] = 2048, hop_length: int = 512, window_fn: str = 'hann_window', wkwargs: Optional[Dict] = None, power: Optional[int] = None, center: bool = True, normalized: bool = True, pad_mode: str = 'constant', onesided: bool = True) -> None


In [3]:
import core.loss as L
names = [n for n in dir(L) if 'sdr' in n.lower()]
print(names)


[]


In [5]:
import pkgutil, core
import core.loss as L
print("Available in core.loss:", [m for m in dir(L) if not m.startswith("_")])

Available in core.loss: ['MultiStemWrapperFromConfig', 'ReImL1Loss', 'ReImL2Loss', 'TimeFreqL1Loss', 'TimeFreqL2Loss', 'TimeFreqSignalNoisePNormRatioLoss', 'snr']


In [1]:
import inspect, core.loss as L
print("SIGNATURE:", inspect.signature(L.MultiStemWrapperFromConfig.__init__))
print("--- SOURCE ---")
print(inspect.getsource(L.MultiStemWrapperFromConfig))

D:\anaconda\envs\ambi\lib\site-packages\lightning_fabric\__init__.py:41: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.


SIGNATURE: (self, name: str, kwargs: Any, modality: str = 'audio') -> None
--- SOURCE ---
class MultiStemWrapperFromConfig(MultiStemWrapper):
    def __init__(self, name: str, kwargs: Any, modality: str = "audio") -> None:
        loss = parse_loss(name, kwargs)
        super().__init__(module=loss, modality=modality)



In [2]:
import inspect
from core.loss._multistem import parse_loss
print(inspect.getsource(parse_loss))

def parse_loss(name: str, kwargs: Dict[str, Any]) -> _Loss:

    for module in [nn.modules.loss, snr, asteroid_losses, asteroid_losses.sdr]:
        if name in module.__dict__:
            return module.__dict__[name](**kwargs)

    raise NameError



In [1]:
import torch
ckpt = torch.load(r"F:\AudioSep\bandit\logs\dnr_subset_test\version_1\checkpoints\last.ckpt", map_location="cpu")
print(ckpt['epoch'])

C:\Users\Manu_Shrivastava\AppData\Local\Temp\ipykernel_26568\3531381995.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(r"F:\AudioSep\bandit\logs\dnr_s

24
